In [3]:
from utils import ut
from model import Proxy, Delay
from matplotlib import get_backend
import matplotlib.pyplot as plt
import matplotlib.dates as md
from datetime import datetime, timedelta
from pybeans import alignment
import pandas as pd

In [4]:
proxies = pd.read_sql(Proxy.__tablename__, ut.conn)
proxies

,id,type,addr,port,remark
0,1,ss,flbgpi.yixuedianzishu.com,58111,动态BGP+IPLC|广港01|3x
1,2,ss,flbgpi.yixuedianzishu.com,58112,动态BGP+IPLC|广港02|3x
2,3,ss,flbgpi.yixuedianzishu.com,58101,动态BGP+IPLC|广港01(原生)|3x
3,4,ss,flbgpi.yixuedianzishu.com,58102,动态BGP+IPLC|广港02(原生)|3x
4,5,ss,flbgpi.yixuedianzishu.com,58103,动态BGP+IPLC|广港03(原生)|3x
5,6,ss,flbgpi.yixuedianzishu.com,58121,动态BGP+IPLC|广新01(原生)|3x
6,7,ss,flbgpi.yixuedianzishu.com,58131,动态BGP+IPLC|广台01(原生)|3x
7,8,ss,flbgpi.yixuedianzishu.com,58151,动态BGP+IPLC|广美01|3x
8,9,vmess,fliplc.getxlx.com,59991,iplc|广港01|1.5x
9,10,vmess,fliplc.getxlx.com,59992,iplc|广港02|1.5x


In [40]:
def query_delay(proxy_id:int):
    return ut.session \
        .query(Delay.proxy_id, Proxy.type, Delay.when, Delay.value) \
        .join(Delay, Proxy.id == Delay.proxy_id) \
        .where(Delay.when > (datetime.now()-timedelta(days = 3))) \
        .order_by(Delay.when)

In [41]:
q = query_delay(1)
df = pd.read_sql(q.statement, q.session.bind, parse_dates=["when"])

2021-07-20 10:06:53,357 INFO sqlalchemy.engine.Engine SELECT proxy_delay.proxy_id, proxy_list.type, proxy_delay.`when`, proxy_delay.value 
FROM proxy_list INNER JOIN proxy_delay ON proxy_list.id = proxy_delay.proxy_id 
WHERE proxy_delay.`when` > %(when_1)s ORDER BY proxy_delay.`when`
2021-07-20 10:06:53,359 INFO sqlalchemy.engine.Engine [generated in 0.01679s] {'when_1': datetime.datetime(2021, 7, 17, 10, 6, 52, 859680)}


In [42]:
df.describe()

,proxy_id,value
count,148204.000000,147040.000000
mean,30.397155,385.482161
std,17.302857,266.354340
min,1.000000,0.000000
25%,15.000000,119.000000
50%,30.000000,364.000000
75%,45.000000,568.000000
max,60.000000,999.000000


In [64]:
df.head(3).T.to_dict().values()
from numpy import float64
round(float64(12.3435), 2)

12.34